In [2]:
from scipy.stats import laplace
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Importa o dataset

In [3]:
dataset = pd.read_csv("adult.csv")
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country,class
0,2,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,1,0,2,United-States,<=50K
1,3,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,0,United-States,<=50K
2,2,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,2,United-States,<=50K
3,3,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,2,United-States,<=50K
4,1,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,2,Cuba,<=50K


## Consultas

* Q1: Média do atributo age
* Q2: Somatório do atributo capital-gain
* Q3: Média do atributo hours-per-week
* Q4: Quantidade de pessoas com income > 50K

In [4]:
def Q1(dataset):
    return dataset.age.mean()

def Q2(dataset):
    return dataset.capitalgain.sum()

def Q3(dataset):
    return dataset.hoursperweek.mean()

def Q4(dataset):
    DatasetProcessado = dataset[dataset["class"] == ">50K"]
    return DatasetProcessado.shape[0]


## Vizinhança


In [5]:
def gerarVizinho(dataset,linhaRemover):
    return dataset.drop([linhaRemover])

def gerarListaVizinhosOtimizado(dataset, f):
    naoMudar = NaoMudar(f)
    unicos = []
    listaTodosVizinhos = []
    for i in range(dataset.shape[0]):
        if(dataset.iloc[i][naoMudar] not in unicos):
            unicos.append(dataset.iloc[i][naoMudar])
            listaTodosVizinhos.append(gerarVizinho(dataset, i))
    return listaTodosVizinhos

In [6]:
def NaoMudar(f):
    if(f==Q1):
        return "age"
    if(f==Q2):
        return "capitalgain"
    if(f==Q3):
        return "hoursperweek"
    if(f==Q4):
        return "capitalgain"

## Sensibilidade

In [7]:
def calcularSensibilidade(dataset, f, sensibilidadeDatasetOriginal):   
    maiorDiferenca = 0
        
    for vizinho in gerarListaVizinhosOtimizado(dataset, f):
        diferenca = abs(sensibilidadeDatasetOriginal - f(vizinho))
        if maiorDiferenca < diferenca:
            maiorDiferenca = diferenca
            
    return maiorDiferenca

## Laplace

In [8]:
def calcularRuido(delta, budget, u):
    return laplace.rvs(loc=u, scale=delta/budget)

## Apresenta Laplace

In [9]:
def apresentarDistribuicao(budget, u, sensibilidade, consulta):
    fig, ax = plt.subplots()
 
    r = laplace.rvs(loc=u, scale=sensibilidade/budget, size=1000)
    ax.hist(r, density=True, histtype='stepfilled', alpha=0.2)  
   
    plt.xlabel('x')
    plt.ylabel('laplace')
    plt.title('Budget = {} - Consulta {}'.format(budget, consulta))

    plt.show()

## Mecanismo

In [19]:
def calcularResposta(dataset, funcoes, budgets, u):
    resultados = []
    sensibilidades = []
    
    for budget in budgets:
        indiceFuncao = 1
        
        for funcao in funcoes:
            sensibilidadeDatasetOriginal = funcao(dataset)
            delta = calcularSensibilidade(dataset, funcao, sensibilidadeDatasetOriginal)
            
            ruido = calcularRuido(delta, budget/len(funcoes), u)
            
            fD = funcao(dataset)
            resultado = fD + ruido
            
#######             apresentarDistribuicao(budget, u, delta, 'Q'+str(indiceFuncao))
            
            print('Budget: {}'.format(budget))
            print('Função: Q{}'.format(indiceFuncao))
            print('Sensibilidade: {}'.format(delta))
            print('F(D): {}'.format(fD))
            print('Ruído: {}'.format(ruido))
            print('Ruído + F(D): {}'.format(resultado))
            print('')
            
            indiceFuncao += 1
            
            sensibilidades.append(delta)
            resultados.append(resultado)
    
    consultas = ['Q1', 'Q2', 'Q3', 'Q4']

    return (budgets, consultas, sensibilidades, resultados)

## Salvando o resultado

In [25]:
# budget, result q1, result q2, result q3, result q4, sens q1, sens q2, sens q3, sens q4
# [budget], [r1-1 r1-2 r1-3 .... r10-4 ], [s1-1 s1-2 s1-3 .... s10-4 ]
def salvarResultados(resultadosLaplace):
    print(resultadosLaplace)
    
    budgets = resultadosLaplace[0]
    consultas = resultadosLaplace[1]
    sensibilidades = resultadosLaplace[2]
    resultados = resultadosLaplace[3]
    atributoResultado = []
    atributoSensibilidade = []
    
    
    
    indiceBudget = 0
    for budget in budgets:
        resultado = []
        sensibilidade = []
        
        indice = (indiceBudget * len(consultas))
#         atributoResultado.append([budget, resultados[indice], resultados[indice + 1], resultados[indice + 2], resultados[indice + 3]])
#         atributoSensibilidade.append([budget, sensibilidades[indice], sensibilidades[indice + 1], sensibilidades[indice + 2], sensibilidades[indice + 3]])
        
        resultado.append(budget)
        sensibilidade.append(budget)
        
        for indiceConsulta in range(len(consultas)):
            resultado.append(resultados[indice + indiceConsulta])
            sensibilidade.append(sensibilidades[indice + indiceConsulta])
        
        atributoResultado.append(resultado)
        atributoSensibilidade.append(sensibilidade)
        indiceBudget += 1
    
    atributoResultadoDataframe =  pd.DataFrame(atributoResultado, columns=['budget', 'result q1', 'result q2', 'result q3', 'result q4'])
    atributoSensibilidadeDataframe =  pd.DataFrame(atributoSensibilidade, columns=['budget2', 'sens q1', 'sens q2', 'sens q3', 'sens q4'])
    resultadoDataframe = pd.concat([atributoResultadoDataframe, atributoSensibilidadeDataframe], axis=1, join='inner')
    resultadoDataframe.drop(columns=['budget2'], inplace=True)
    resultadoDataframe.to_csv('resultado.csv', index=False)

In [ ]:
a = pd.DataFrame([[1, 2, 3], [4, 5, 6], [7, 8, 9]], columns = ['T1', 'T2', 'T3'])
b = pd.DataFrame([[11, 22, 3], [44, 55, 6], [77, 88, 9]], columns = ['T4', 'T5', 'T6'])
c = pd.concat([a, b], axis=1, join='inner')
c.drop(columns=['T6'], inplace=True)
c

In [26]:
def main():
    budgets = [0.1, 1, 10]
    funcoes = [Q1, Q2, Q3, Q4]
    u = 0
    resultados = calcularResposta(dataset, funcoes, budgets, u)
    salvarResultados(resultados)

In [27]:
main()

Budget: 0.1
Função: Q1
Sensibilidade: 4.5636285693984036e-05
F(D): 1.7710781704270915
Ruído: 0.0033260662383417495
Ruído + F(D): 1.7744042366654331

Budget: 0.1
Função: Q2
Sensibilidade: 4
F(D): 9784
Ruído: -14.571385407346739
Ruído + F(D): 9769.428614592653

Budget: 0.1
Função: Q3
Sensibilidade: 4.19586378328507e-05
F(D): 1.9506981696081243
Ruído: -0.002551895390161096
Ruído + F(D): 1.9481462742179632

Budget: 0.1
Função: Q4
Sensibilidade: 1
F(D): 11687
Ruído: 24.16139611604804
Ruído + F(D): 11711.161396116047

Budget: 1
Função: Q1
Sensibilidade: 4.5636285693984036e-05
F(D): 1.7710781704270915
Ruído: 1.1529481959852366e-06
Ruído + F(D): 1.7710793233752875

Budget: 1
Função: Q2
Sensibilidade: 4
F(D): 9784
Ruído: -15.902118945310722
Ruído + F(D): 9768.09788105469

Budget: 1
Função: Q3
Sensibilidade: 4.19586378328507e-05
F(D): 1.9506981696081243
Ruído: 4.2812823258067765e-05
Ruído + F(D): 1.9507409824313824

Budget: 1
Função: Q4
Sensibilidade: 1
F(D): 11687
Ruído: 1.6352813850379964
Ruíd